# T4 Combine Detection Results

- import the t1 inferenced point clouds
- import the t3 doors 
- export graphs

## LIBRARIES

In [1]:

#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t1_utils as t1
import utils.t2_utils as t2
import utils.t4_utils as t4

import json

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[2]

print(path)
input_folder=path/'data'/'t2'/'test' 
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t4'/ 'test'
os.makedirs(output_folder, exist_ok=True)

#parameters
resolution=0.05
min_cluster_points=1000
eps=0.5
size=[12,12,100] #size wall boxes
threshold_column_height=1.5#m
threshold_wall_dim=0.5#m
threshold_wall_verticallity=0.2# angle to horizontal
threshold_door_dim=1.6#m

c:\Users\u0094523\OneDrive - KU Leuven\2024-05 CVPR scan-to-BIM challenge


### Import classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


## CREATE INITIAL OBJECT PCDS

In [6]:
files=utl.get_list_of_files(input_folder, '.laz')

In [8]:
for f in files[:1]:#, files_t2[:1], files_t3[:1]):
    objectNodes=[]
    
    # check if las/pcd variable is already defined    
    print(f'processing {ut.get_filename(f)}...')      
    las = laspy.read(f) if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las,getNormals=True) if 'pcd' not in globals() else pcd
    
    
    #seperate initial objects
    for c in class_dict['classes']:
        
        ###------------------------FLOORS CEILINGS--------------------------------------------
        if c['id'] in [0,1]: #floors,ceilings CORRECT
            idx=np.where((las['classes']==c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            
            #retrieve objects from planar clusters
            clustered_pcds,clustered_planes=t4.split_point_cloud_in_planar_clusters(class_pcd,
                                                                                    sample_resolution=resolution,
                                                                                    distance_threshold=0.05, 
                                                                                    min_inliers=1000,
                                                                                    eps=0.5,
                                                                                    min_cluster_points=200)
            
            for i,n in enumerate(clustered_pcds):
                objectNodes.append(PointCloudNode(resource=n,
                                            class_id=c['id'],
                                            class_name=c['name'],
                                            object_id=i+1+c['id']*100, #+1 because 0 is clutter
                                            plane=clustered_planes[i],
                                            color=ut.random_color(),
                                            name=ut.get_filename(f)+'_'+c['name']+'_'+str(i+1+c['id']*100)))       
            print( c['name'], f': {len(clustered_pcds)} clusters found')       
        
        ###------------------------WALLS--------------------------------------------
        if c['id'] in [2]: 
            counter=0
            idx=np.where((las['classes']==c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            
            #split into boxes (otherwise you will get horizontal planes mostly)
            min_bound=pcd.get_min_bound()
            max_bound=pcd.get_max_bound()            
            box=o3d.geometry.AxisAlignedBoundingBox(min_bound=np.array([min_bound[0]-5,min_bound[1]-5,min_bound[2]-5]),
                                        max_bound=np.array([max_bound[0]+5,max_bound[1]+5,max_bound[2]+5]))
            boxes,names=gmu.divide_box_in_boxes(box,size=size)
            
            # select indices per boxes
            sub_pcds=[]
            for box,name in zip(boxes,names):
                idxLists=box.get_point_indices_within_bounding_box(class_pcd.points)
                sub_pcd=class_pcd.select_by_index(idxLists)
            
                #retrieve objects from planar clusters
                if len(np.asarray(sub_pcd.points))<min_cluster_points:
                    continue
                clustered_pcds,clustered_planes=t4.split_point_cloud_in_planar_clusters(sub_pcd,
                                                                                    sample_resolution=0.03,
                                                                                    distance_threshold=0.03, 
                                                                                    min_inliers=200,
                                                                                    eps=0.5,
                                                                                    min_cluster_points=200)
 
                if len(clustered_pcds)!=0:
                    for p,pl in zip(clustered_pcds,clustered_planes):
                        counter+=1
                        objectNodes.append(PointCloudNode(resource=p,
                                                    class_id=c['id'],
                                                    class_name=c['name'],
                                                    object_id=c['id']*100+counter, 
                                                    plane=pl,
                                                    color=ut.random_color(),
                                                    name=ut.get_filename(f)+'_'+c['name']+'_'+str(c['id']*100+counter)))       
            print( c['name'], f': {counter} clusters found')  
            
        # ###------------------------COLUMNS--------------------------------------------
        if c['id'] in [3]:
            idx = np.where((las['classes'] == c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            object_labels=las['objects'][idx]
            
            object_pcds=[]
            for i in np.unique(object_labels): 
                if i==0:
                    potential_object_pcds=t4.split_point_cloud_by_dbscan(class_pcd.select_by_index(np.where(object_labels==i)[0]),eps=eps,min_cluster_points=min_cluster_points)
                    #retain only clusters with sufficient height
                    object_pcds.extend([p for p in potential_object_pcds])
                else:           
                    object_pcds.append(class_pcd.select_by_index(np.where(object_labels==i)[0]))
            
            for i,p in enumerate(object_pcds):    
                objectNodes.append(PointCloudNode(resource=p,
                                            class_id=c['id'],
                                            class_name=c['name'],
                                            object_id=c['id']+i,
                                            color=ut.random_color(),
                                            name=ut.get_filename(f)+'_'+c['name']+'_'+str(c['id']*100+i)))
            print( c['name'], f': {len(object_pcds)} clusters found')       
            
        # ###------------------------DOORS--------------------------------------------
        if c['id'] in [4]: 
            idx = np.where((las['classes'] == c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            object_labels=las['objects'][idx]
            
            object_pcds=[]
            for i in np.unique(object_labels): 
                if i==0:
                    object_pcds.extend(t4.split_point_cloud_by_dbscan(class_pcd.select_by_index(np.where(object_labels==i)[0]),eps=eps,min_cluster_points=min_cluster_points))
                else:           
                    object_pcds.append(class_pcd.select_by_index(np.where(object_labels==i)[0]))
            
            for i,p in enumerate(object_pcds):    
                objectNodes.append(PointCloudNode(resource=p,
                                            class_id=c['id'],
                                            class_name=c['name'],
                                            object_id=c['id']+i,
                                            color=ut.random_color(),
                                            name=ut.get_filename(f)+'_'+c['name']+'_'+str(c['id']*100+i)))
            print( c['name'], f': {len(object_pcds)} clusters found')      
        else:
            idx=np.where((las['classes']==c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            thrashNode=PointCloudNode(resource=pcd,
                                            class_id=c['id'],
                                            class_name=c['name'],
                                            color=ut.random_color(),
                                            name=ut.get_filename(f)+'_unassigned')

    print(f'Created {len(objectNodes)} PointCloudNodes created from {ut.get_filename(f)}')


processing 08_ShortOffice_01_F1_small_pred...
Function fit_planes took 3.1143 seconds to execute.
floors : 4 clusters found
Function fit_planes took 11.6154 seconds to execute.
ceilings : 9 clusters found
Function fit_planes took 7.7359 seconds to execute.
Function fit_planes took 7.1171 seconds to execute.
Function fit_planes took 4.0049 seconds to execute.
Function fit_planes took 7.3100 seconds to execute.
Function fit_planes took 6.9514 seconds to execute.
Function fit_planes took 3.2288 seconds to execute.
Function fit_planes took 2.3468 seconds to execute.
Function fit_planes took 8.2539 seconds to execute.
Function fit_planes took 1.6029 seconds to execute.
Function fit_planes took 4.8709 seconds to execute.
Function fit_planes took 3.6841 seconds to execute.
Function fit_planes took 3.7587 seconds to execute.
walls : 257 clusters found
columns : 42 clusters found
doors : 7 clusters found
Created 319 PointCloudNodes created from 08_ShortOffice_01_F1_small_pred


## VISUALIZE

In [16]:
{key:value for key, value in objectNodes[1].__dict__.items() if not key.startswith('__') and not callable(key)}              

{'_e57Index': 0,
 'pointCount': 14959,
 'e57XmlPath': None,
 '_cartesianBounds': array([-2.28800000e+01, -1.35000000e+01,  9.39000000e+00,  1.91700000e+01,
        -1.00000000e-02,  5.28571429e-02]),
 '_orientedBounds': array([[-2.00641522e+01,  2.22818819e+01,  4.63756496e-02],
        [-1.12199082e+01,  1.50613484e+01,  5.91188445e-02],
        [-2.52897033e+01,  1.58812512e+01,  5.27173734e-02],
        [-2.00641253e+01,  2.22817896e+01, -2.45775967e-02],
        [-1.64454323e+01,  8.66062540e+00, -5.49267805e-03],
        [-2.52896764e+01,  1.58811589e+01, -1.82358730e-02],
        [-1.12198813e+01,  1.50612561e+01, -1.18344018e-02],
        [-1.64454592e+01,  8.66071767e+00,  6.54605683e-02]]),
 '_orientedBoundingBox': OrientedBoundingBox: center: (-18.2548, 15.4713, 0.0204415), extent: 11.4174, 8.26284, 0.0709533),
 '_subject': rdflib.term.URIRef('file:///08_ShortOffice_01_F1_small_pred_floors_2'),
 '_graph': None,
 '_graphPath': None,
 '_path': None,
 '_name': '08_ShortOffice_01

In [20]:
pcd_slice=t2.slice_point_cloud(pcd, -100, pcd.get_center()[2])

In [62]:
joined_pcd=gmu.join_geometries([p.resource.paint_uniform_color(ut.random_color()) for p in objectNodes])
o3d.visualization.draw_geometries([joined_pcd,gmu.sample_geometry(class_pcd)[0]])

## PROCESS WALLS

In [63]:
# wall planes should be near vertical!
wallNodes=[n for n in objectNodes if n.class_id == 2]
print(f'Initial number of nodes:{len(wallNodes)} ')
newWallNodes=[]

for n in wallNodes:
    dim=n.resource.get_oriented_bounding_box().extent
    # check verticality and dimensions
    if (np.abs(n.plane[2])<threshold_wall_verticallity) & (dim[0]>threshold_wall_dim) & (dim[1]>threshold_wall_dim):
        newWallNodes.append(n)
    else:
        thrashNode.resource+=n.resource

print(f'Retained {len(newWallNodes)} merged column nodes.')    

Initial number of nodes:238 
Retained 153 merged column nodes.


In [64]:
# joined_pcd1=gmu.join_geometries([p.resource.paint_uniform_color([1,0,0]) for p in wallNodes if p not in newWallNodes])

# o3d.visualization.draw_geometries([joined_pcd1,gmu.sample_geometry(class_pcd)[0]])

## PROCESS DOORS

In [67]:
# wall planes should be near vertical!
doorPcdNodes=[n for n in objectNodes if n.class_id == 4]
print(f'Initial number of nodes:{len(doorPcdNodes)} ')
newDoorPcdNodes=[]

for n in doorPcdNodes:
    dim=n.resource.get_oriented_bounding_box().extent
    # check verticality and dimensions
    if (dim[0]>threshold_door_dim) & (dim[1]>threshold_door_dim):
        newDoorPcdNodes.append(n)
    else:
        thrashNode.resource+=n.resource

print(f'Retained {len(newDoorPcdNodes)} merged column nodes.')    

Initial number of nodes:5 
Retained 1 merged column nodes.


## PROCESS COLUMNS

In [71]:
threshold_column_points=100
threshold_clustering_distance=0.5
threshold_column_height=1
threshold_column_verticallity=0.2

columnPcdNodes=[n for n in objectNodes if n.class_id == 3]
print(f'Initial number of nodes:{len(columnPcdNodes)} ')

for n in columnPcdNodes: 
    
    #scrap horizontal columns
    n.resource=n.resource.select_by_index(np.where(np.asarray(n.resource.normals)[:,2]<threshold_column_verticallity)[0])
    n.get_oriented_bounding_box()
    
    if len(np.asarray(n.resource.points))<threshold_column_points or  (p.get_axis_aligned_bounding_box().max_bound[2]-p.get_axis_aligned_bounding_box().min_bound[2])>threshold_column_height:
        columnPcdNodes.pop(n)
        thrashNode.resource+=n.resource
    else:
        for m in columnPcdNodes:
            if n!=m and len(np.asarray(n.resource.points))<len(np.asarray(m.resource.points)) and np.linalg.norm(n.cartesianTransform[0:3,3] - m.cartesianTransform[0:3,3]) < threshold_clustering_distance:
                m.resource+=n.resource
                columnPcdNodes.pop(n)
                break
print(f'Retained {len(columnPcdNodes)} merged column nodes.')

Initial number of nodes:48 
Retained 48 merged column nodes.


In [72]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(n.color) for n in columnPcdNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd])

## OUTPUTS

### export graphs

In [12]:
total_pcd_nodes=[n for n in objectNodes if n.class_id in [0,1,2]]+doorPcdNodes+columnPcdNodes

In [13]:
tl.nodes_to_graph(total_pcd_nodes,
                graphPath=str(output_folder/f'{ut.get_filename(t1)}_pcdgraph.ttl'),
                save=True)

<Graph identifier=Nbdce8cadde464f199ab7743101b6ca42 (<class 'rdflib.graph.Graph'>)>

### export point cloud


In [14]:
joined_pcd=gmu.join_geometries([n.resource for n  in total_pcd_nodes])
print(joined_pcd)

PointCloud with 401102 points.


In [15]:
#obtain labels
labels_segmentation=[]
labels_objects=[]

for i,n in enumerate(total_pcd_nodes):
    length=len(np.asarray(n.resource.points))
    labels_segmentation.extend(list(np.full(length,n.class_id)))
    labels_objects.extend(list(np.full(length,n.object_id)))  

labels_classes=np.array(labels_segmentation)
labels_objects=np.array(labels_objects)

In [16]:
#interpolate labels
indices,distances=gmu.compute_nearest_neighbors(np.asarray(las.xyz),np.asarray(joined_pcd.points))

In [103]:
labels_c=np.full((las.xyz.shape[0],1),255)
labels_o=np.full((las.xyz.shape[0],1),0)

In [102]:
inliers.shape
outliers.shape

(3472028,)

In [104]:
#inliers
inliers=np.where(distances<resolution)[0]

#modify the labels_c array with the labels_classes array based on the inliers
labels_c[inliers]=labels_classes[inliers]
labels_o[inliers]=labels_objects[indices]


IndexError: index 401938 is out of bounds for axis 0 with size 401938

In [ ]:
#outliers
outliers=np.where(distances>=resolution)[0]
labels_c[outliers]=labels_classes[outliers]
labels_o[outliers]=labels_objects[outliers]

In [ ]:
# create new las
header = laspy.LasHeader(point_format=3, version="1.2")
las = laspy.LasData(header)
las.xyz=np.asarray(total_pcd.points)
las.red=(np.asarray(total_pcd.colors)[:,0]*65535).astype(np.uint16)
las.green=(np.asarray(total_pcd.colors)[:,1]*65535).astype(np.uint16)
las.blue=(np.asarray(total_pcd.colors)[:,2]*65535).astype(np.uint16)

gmu.las_add_extra_dimensions(las,(labels_segmentation[:,0],labels_objects[:,0],),['classes','objects'],['uint8','uint16'])
print(list(las.point_format.dimension_names))

In [ ]:
las.write(str(output_folder/f'{ut.get_filename(t1)}_t4.laz'))
print(str(output_folder/f'{ut.get_filename(t1)}_t4.laz'))